# Feature Engineering Exercises

In [1]:
import pandas as pd
import numpy as np
import wrangle
import warnings
from pydataset import data
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

#### 1. Load the `tips` dataset 
    a. Create a column named price_per_person. This should be the total bill divided by the party size. \
    b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? \
    c. Use select k best to select the top 2 features for predicting tip amount. What are they? \
    d. Use recursive feature elimination to select the top 2 features for tip amount. What are they? \
    e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [2]:
tips = data('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2


a. Create a column named price_per_person. This should be the total bill divided by the party size. 

In [3]:
tips['price_per_person'] = tips["total_bill"] / tips["size"]

In [4]:
cleanup_nums = {"sex": {"Male": 1, "Female": 0}, "smoker": {"Yes": 1, "No": 0}}

In [5]:
tips = tips.replace(cleanup_nums)

In [6]:
tips = pd.get_dummies(tips, columns=['day','time','size'])

In [7]:
tips.head()

,total_bill,tip,sex,smoker,price_per_person,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch,size_1,size_2,size_3,size_4,size_5,size_6
1,16.99,1.01,0,0,8.495000,0,0,1,0,1,0,0,1,0,0,0,0
2,10.34,1.66,1,0,3.446667,0,0,1,0,1,0,0,0,1,0,0,0
3,21.01,3.50,1,0,7.003333,0,0,1,0,1,0,0,0,1,0,0,0
4,23.68,3.31,1,0,11.840000,0,0,1,0,1,0,0,1,0,0,0,0
5,24.59,3.61,0,0,6.147500,0,0,1,0,1,0,0,0,0,1,0,0


b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? 

I would predict `total_bill`, `sex`, and `size` would be most important for predicting the tip amount.

c. Use select k best to select the top 2 features for predicting tip amount. What are they? 

In [8]:
tips.smoker.value_counts()

0    151
1     93
Name: smoker, dtype: int64

In [9]:
tips.shape

(244, 17)

In [10]:
x_train = tips.drop(columns=['tip'], axis=1)
y_train = tips['tip']

In [11]:
x_train.head()

,total_bill,sex,smoker,price_per_person,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch,size_1,size_2,size_3,size_4,size_5,size_6
1,16.99,0,0,8.495000,0,0,1,0,1,0,0,1,0,0,0,0
2,10.34,1,0,3.446667,0,0,1,0,1,0,0,0,1,0,0,0
3,21.01,1,0,7.003333,0,0,1,0,1,0,0,0,1,0,0,0
4,23.68,1,0,11.840000,0,0,1,0,1,0,0,1,0,0,0,0
5,24.59,0,0,6.147500,0,0,1,0,1,0,0,0,0,1,0,0


In [12]:
kbest = SelectKBest(f_regression, k=2)
kbest.fit(x_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fd62f9e8040>)

In [13]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=x_train.columns)
kbest_results

,p,f
total_bill,6.692471e-34,203.357723
sex,1.664562e-01,1.926155
smoker,9.265932e-01,0.008506
price_per_person,2.502102e-08,33.213257
day_Fri,3.883700e-01,0.746727
day_Sat,9.654161e-01,0.001884
day_Sun,5.094012e-02,3.848390
day_Thur,1.353240e-01,2.245302
time_Dinner,5.780153e-02,3.633815
time_Lunch,5.780153e-02,3.633815


In [14]:
x_train_transformed = pd.DataFrame(
    kbest.transform(x_train),
    index=x_train.index,
    columns=x_train.columns[kbest.get_support()]
)
x_train_transformed.head()

,total_bill,size_2
1,16.99,1.0
2,10.34,0.0
3,21.01,0.0
4,23.68,1.0
5,24.59,0.0


The top 2 features selected by KBest are:
- `total_bill`
- `size_2`

d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [15]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(x_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [16]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=x_train.columns)

,rfe_ranking
total_bill,4
sex,13
smoker,8
price_per_person,6
day_Fri,5
day_Sat,11
day_Sun,3
day_Thur,10
time_Dinner,9
time_Lunch,12


In [17]:
x_train_transformed = pd.DataFrame(
    rfe.transform(x_train),
    index=x_train.index,
    columns=x_train.columns[rfe.support_]
)
x_train_transformed.head()

,size_2,size_6
1,1.0,0.0
2,0.0,0.0
3,0.0,0.0
4,1.0,0.0
5,0.0,0.0


The top 2 features selected by RFE are:
- `size_2`
- `size_6`

e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

The features from SelectKBest are being chosen based on correlation with the target variable, while the features from RFE are selected based on the performance of a linear regression model. These two approaches are different, which is why they may produce different results.

#### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. 

In [18]:
#Define the SelectKBest function
def select_kbest(X, y, k):
    
    f_selector = SelectKBest(f_regression, k=k)
    
    f_selector.fit(X, y)
    
    mask = f_selector.get_support()
    
    return list(X.columns[mask])

In [19]:
#Verify it works
select_kbest(x_train, y_train, 5)

['total_bill', 'price_per_person', 'size_2', 'size_4', 'size_6']

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [20]:
#Define the RFE function
def rfe(X, y, k):
    
    lm = LinearRegression()
    
    rfe = RFE(lm, n_features_to_select=k)
    
    X_rfe = rfe.fit_transform(X, y)
    
    mask = rfe.support_
    
    return list(X.columns[mask])

In [21]:
#Test it
rfe(x_train, y_train, 5)

['total_bill', 'day_Sun', 'size_2', 'size_5', 'size_6']

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [22]:
#Load in the swiss dataset
swiss = data('swiss')

swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [23]:
#Create X and y dataframes
X = swiss.drop(columns=['Fertility'])
y = swiss['Fertility']

In [24]:
#Create a list of columns
X_columns = list(X.columns)

In [25]:
#Instantiate a scaler and fit/transform the columns
mms2 = MinMaxScaler()

X[X_columns] = mms2.fit_transform(X[X_columns])

X.head()

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,0.178531,0.352941,0.211538,0.079816,0.721519
Delemont,0.496045,0.088235,0.153846,0.845069,0.721519
Franches-Mnt,0.435028,0.058824,0.076923,0.932550,0.594937
Moutier,0.398870,0.264706,0.115385,0.323148,0.601266
Neuveville,0.477966,0.411765,0.269231,0.030761,0.620253


In [26]:
#Run the select k best function
select_kbest(X, y, 3)

['Examination', 'Education', 'Catholic']

In [27]:
#Run the rfe function
rfe(X, y, 3)

['Agriculture', 'Education', 'Infant.Mortality']

In [28]:
X_2 = swiss.drop(columns=['Fertility'])
y_2 = swiss['Fertility']

In [29]:
X_2.head()

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,17.0,15,12,9.96,22.2
Delemont,45.1,6,9,84.84,22.2
Franches-Mnt,39.7,5,5,93.40,20.2
Moutier,36.5,12,7,33.77,20.3
Neuveville,43.5,17,15,5.16,20.6


In [30]:
select_kbest(X_2, y_2, 3)

['Examination', 'Education', 'Catholic']

In [31]:
rfe(X_2, y_2, 3)

['Examination', 'Education', 'Infant.Mortality']

# Additional Work (not part of exercises)

##### Creating 'for loop's, iterating through values of k, for kbest and rfe

In [32]:
def kbest_loop(x_train, y_train, ):
    

IndentationError: expected an indented block (2911545075.py, line 2)

In [ ]:
feature_sets_to_model = []
    
for i in range(1,10)
    # Create kbest object
    feature_set = SelectKBest(f_regression, k=i)
    feature_sets_to_model.append(feature_set)
        
        
        

In [ ]:
eval_dict = {}

for feature_set in feature_sets_to_model:
    

In [ ]:
        # Fit object to df
        kbest.fit(x_train, y_train)
        
        # Use the model
        kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=x_train.columns)
        kbest_results
        
        #
        x_train_transformed = pd.DataFrame(
        kbest.transform(x_train),
        index=x_train.index,
        columns=x_train.columns[kbest.get_support()]
        )
        x_train_transformed.head()
        
        output = {
            "k": i,
            "p_value":
            "f_value":
        }

In [ ]:
feature_sets_to_model = []